## Azure Databricks / Azure Machine Learning Sample - Model Endpoint Testing

Sample notebook showcasing how to submit multiple requests to an Azure Machine Learning Managed Online Endpoint for real-time data scoring.

### Import required packages

In [0]:
import pandas as pd
import random
import json
import requests
from concurrent.futures import ThreadPoolExecutor

### Retrieve endpoint URI and key from Databricks secret store

In [0]:
aml_endpoint_uri = dbutils.secrets.get(scope="amlsecretscope",key="amlendpointuri")
aml_endpoint_key = dbutils.secrets.get(scope="amlsecretscope",key="amlendpointkey")

### Sample data + modification function

Example of data in the appropriate format for endpoint submission. Helper function modify numerical values slightly prior to scoring.

In [0]:
def modify_data(sample_data):
    for i in range(len(sample_data['input_data']['data'])):
        for j in range(len(sample_data['input_data']['data'][i])):
            if isinstance(sample_data['input_data']['data'][i][j], (int, float)):
                sample_data['input_data']['data'][i][j] = sample_data['input_data']['data'][i][j] * random.uniform(0.75, 1.25)
    return sample_data

sample_data = {
  "input_data": {"index": [0,1,2],
  "columns": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
  "data": [[0.00538306037424807,
    -0.044641636506989,
    -0.0482406250171634,
    -0.0125563519424068,
    0.00118294589619092,
    -0.00663740127664067,
    0.0633666506664982,
    -0.0394933828740919,
    -0.0514005352605825,
    -0.0590671943081523],
   [0.0126481372762872,
    0.0506801187398187,
    0.000260918307477141,
    -0.0114087283893043,
    0.0397096259258226,
    0.0572448849284239,
    -0.0397192078479398,
    0.0560805201945126,
    0.024052583226893,
    0.0320591578182113],
   [0.0380759064334241,
    0.0506801187398187,
    0.00888341489852436,
    0.0425295791573734,
    -0.0428475455662452,
    -0.0210422305189592,
    -0.0397192078479398,
    -0.00259226199818282,
    -0.0181182673078967,
    0.00720651632920303]]}
}

modify_data(sample_data)

### Endpoint request submission

Sample code for submitting multiple requests to the endpoint

In [0]:
def send_request(url, data, headers):
    mod_data = modify_data(data)
    response = requests.post(url, data=json.dumps(mod_data), headers=headers)
    print(response)
    try:
        print(response.json())
    except Exception as e:
        pass
    return response

urls = [aml_endpoint_uri] * 100
data = sample_data
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ aml_endpoint_key)}


with ThreadPoolExecutor(max_workers=1) as executor:
    results = list(executor.map(send_request, urls, [data] * len(urls), [headers] * len(urls)))